In [14]:
%pip install transformers fugashi ipadic sentencepiece detach

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from transformers import MLukeTokenizer, LukeModel
import torch

In [16]:
class SentenceLukeJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = MLukeTokenizer.from_pretrained(model_name_or_path)
        self.model = LukeModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        return torch.stack(all_embeddings)

In [17]:
MODEL_NAME = "sonoisa/sentence-luke-japanese-base-lite"
model = SentenceLukeJapanese(MODEL_NAME)

c:\WorkSpace\Practice\env2\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
import numpy as np

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [19]:
s1 = "日本の風土の特質が日本の歴史と文化に大きな影響を与える。"
s2 = "島国の日本は、四周を海にかこまれ世界から隔てられている。"
s3 = "この地理的条件によって、日本は外敵の侵略と異民族の支配がなく、周囲から文化や技術などを吸収し、「島国」の中でそれを融和させて、独自の文化を磨きあげ築きあげてきたのである。"
s4 = "外国との交通が海に隔てられ、発信的な文化型は形成しにくいので、自ら外国の文化を積極的に受けいれる受信的な文化型を形成してきた。"
s5 = "日本文化の本質は受信文化である。"
s6 = "外来文化の受信能力が世界一である日本から発信することはほとんどない。"
s7 = "日本は外来文化を選択して受信するだけにとどまらず、さらに受信した外来文化を巧みに融合して日本化する。"
s8 = "古代朝鮮や中国から漢字や儒教、仏教、道教など宗教思想を導入し、近代欧米の新文明を吸収し、更にそれを自国に適応しようと、工夫に努めた。"
s9 = "受信文化の特徴として、選択の可能性があること。"
s10 = "日本は海に守られて外来侵略と異民族の支配がないため、受け入れる側の意識や都合を無視されるような押し付け、強制的な外来文化の受信は全くない。"
s = model.encode([s1, s2, s3, s4, s5, s6, s7, s8, s9, s10])

In [25]:
print(f's1 vs s2:{cos_sim(s[0].detach(), s[1].detach())}')
print(f's2 vs s3:{cos_sim(s[1].detach(), s[2].detach())}')
print(f's3 vs s4:{cos_sim(s[2].detach(), s[3].detach())}')
print(f's4 vs s5:{cos_sim(s[3].detach(), s[4].detach())}')
print(f's5 vs s6:{cos_sim(s[4].detach(), s[5].detach())}')
print(f's6 vs s7:{cos_sim(s[5].detach(), s[6].detach())}')
print(f's7 vs s8:{cos_sim(s[6].detach(), s[7].detach())}')
print(f's8 vs s9:{cos_sim(s[7].detach(), s[8].detach())}')
print(f's9 vs s10:{cos_sim(s[8].detach(), s[9].detach())}')

s1 vs s2:0.34205830097198486
s2 vs s3:0.5452121496200562
s3 vs s4:0.6079327464103699
s4 vs s5:0.44962888956069946
s5 vs s6:0.46150654554367065
s6 vs s7:0.532287061214447
s7 vs s8:0.6761888861656189
s8 vs s9:0.24020744860172272
s9 vs s10:0.22214150428771973
